# Fig S3

Oxygen consumption in response to both glucose stimulation and chemical agents

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: GlcConst, VmaxPDH, pHleak, VmaxF1, VmaxETC, J_ANT, J_O2
using MitochondrialDynamics: G3P, Pyr, NADH_c, NADH_m, Ca_c, Ca_m, ΔΨm, ATP_c, ADP_c, degavg, t
using MitochondrialDynamics.Utils: second, μM, mV, mM, Hz, minute

using Plots
pyplot()
Plots.pyrcparams["font.size"] = 14
Plots.pyrcparams["font.sans-serif"] = "Arial"
Plots.pyrcparams["font.family"] = "sans-serif"

In [ ]:
add_glucose(t) = ifelse(t<20minute, 5mM, 20mM)
add_oligomycin(t) = ifelse(t<40minute, 1.0, 0.05)
add_rotenone(t) = ifelse(t<60minute, 1.0, 0.05)
add_fccp(t) = ifelse(t<60minute, 1.0, 10.0)

@register_symbolic add_glucose(t) 
@register_symbolic add_oligomycin(t) 
@register_symbolic add_rotenone(t)
@register_symbolic add_fccp(t)

In [ ]:
tend = 80minute
@named sys1 = make_model(;
    glcrhs = add_glucose(t),
    rf1 = add_oligomycin(t),
    rhleak = add_fccp(t)
    )
prob1 = ODEProblem(sys1, DEFAULT_U0, tend)

In [ ]:
idxGlc = findfirst(isequal(GlcConst), parameters(sys1))
idxVmaxPDH = findfirst(isequal(VmaxPDH), parameters(sys1))
idxpHleak = findfirst(isequal(pHleak), parameters(sys1))
idxVmaxF1 = findfirst(isequal(VmaxF1), parameters(sys1))
idxVmaxETC = findfirst(isequal(VmaxETC), parameters(sys1))

function make_dm_prob(prob; rPDH=0.5, rETC=0.75, rHL=1.4, rF1=0.5)
    newparams = copy(prob.p)
    newparams[idxVmaxPDH] = prob.p[idxVmaxPDH] * rPDH
    newparams[idxpHleak] = prob.p[idxpHleak] * rHL
    newparams[idxVmaxF1] = prob.p[idxVmaxF1] * rF1
    newparams[idxVmaxETC] = prob.p[idxVmaxETC] * rETC
    return remake(prob, p=newparams)
end

prob1DM = make_dm_prob(prob1; rPDH=0.5, rETC=0.75, rHL=1.4, rF1=0.5)

In [ ]:
tstops = [20minute, 40minute, 60minute]

sol1 = solve(prob1; tstops=tstops)
sol1DM = solve(prob1DM; tstops=tstops)

In [ ]:
ts = range(0.0, tend, length=301)

g3p = [sol1.(ts, idxs=G3P) sol1DM.(ts, idxs=G3P)] .* 1000
pyr = [sol1.(ts, idxs=Pyr) sol1DM.(ts, idxs=Pyr)] .* 1000
nadh_c = [sol1.(ts, idxs=NADH_c) sol1DM.(ts, idxs=NADH_c)] .* 1000
nadh_m = [sol1.(ts, idxs=NADH_m) sol1DM.(ts, idxs=NADH_m)] .* 1000
atp_c = [sol1.(ts, idxs=ATP_c) sol1DM.(ts, idxs=ATP_c)] .* 1000
adp_c = [sol1.(ts, idxs=ADP_c) sol1DM.(ts, idxs=ADP_c)] .* 1000
td = atp_c ./ adp_c
ca_c = [sol1.(ts, idxs=Ca_c) sol1DM.(ts, idxs=Ca_c)] .* 1000
ca_m = [sol1.(ts, idxs=Ca_m) sol1DM.(ts, idxs=Ca_m)] .* 1000
dpsi = [sol1.(ts, idxs=ΔΨm) sol1DM.(ts, idxs=ΔΨm)] .* 1000
deg = [sol1.(ts, idxs=degavg) sol1DM.(ts, idxs=degavg)]

In [ ]:
tsm = ts ./ 60
labels = ["Baseline" "Diabetic"]
pl1 = plot(tsm, g3p, label=labels, ylabel="G3P (μM)", title="A", titlelocation=:left)
pl2 = plot(pyr, label=labels, ylabel="Pyruvate (μM)", title="B", titlelocation=:left)
pl3 = plot(tsm, nadh_c, label=labels, ylabel="Cytosolic NADH (μM)", title="C", titlelocation=:left)
pl4 = plot(tsm, nadh_m, label=labels, ylabel="Mitochondrial NADH (μM)", title="D", titlelocation=:left)
pl5 = plot(tsm, ca_c, label=labels, ylabel="Cytosolic Calcium (μM)", title="E", titlelocation=:left)
pl6 = plot(tsm, ca_m, label=labels, ylabel="Mitochondrial Calcium (μM)", title="F", titlelocation=:left)
pl7 = plot(tsm, td, label=labels, ylabel="ATP/ADP ratio", title="G", titlelocation=:left)
pl8 = plot(tsm, dpsi, label=labels, ylabel="Mitochodrial membrane potential (mV)", title="H", titlelocation=:left)
pl9 = plot(tsm, deg, label=labels, ylabel="Average Node Degree", title="I", titlelocation=:left)

plot(pl1, pl2, pl3, pl4, pl5, pl6, pl7, pl8, pl9; size=(2000, 2000))

In [ ]:
plot(sol1, vars=[J_O2])
plot!(sol1DM, vars=[J_O2])

In [ ]:
@named sys2 = make_model(;
    glcrhs = add_glucose(t),
    rf1 = add_oligomycin(t),
    retc = add_rotenone(t)
    )

prob2 = ODEProblem(sys2, DEFAULT_U0, tend)
prob2DM = make_dm_prob(prob2; rPDH=0.5, rETC=0.75, rHL=1.4, rF1=0.5)

In [ ]:
sol2 = solve(prob2; tstops=tstops)
sol2DM = solve(prob2DM; tstops=tstops)

In [ ]:
g3p = [sol2.(ts, idxs=G3P) sol2DM.(ts, idxs=G3P)] .* 1000
pyr = [sol2.(ts, idxs=Pyr) sol2DM.(ts, idxs=Pyr)] .* 1000
nadh_c = [sol2.(ts, idxs=NADH_c) sol2DM.(ts, idxs=NADH_c)] .* 1000
nadh_m = [sol2.(ts, idxs=NADH_m) sol2DM.(ts, idxs=NADH_m)] .* 1000
atp_c = [sol2.(ts, idxs=ATP_c) sol2DM.(ts, idxs=ATP_c)] .* 1000
adp_c = [sol2.(ts, idxs=ADP_c) sol2DM.(ts, idxs=ADP_c)] .* 1000
td = atp_c ./ adp_c
ca_c = [sol2.(ts, idxs=Ca_c) sol2DM.(ts, idxs=Ca_c)] .* 1000
ca_m = [sol2.(ts, idxs=Ca_m) sol2DM.(ts, idxs=Ca_m)] .* 1000
dpsi = [sol2.(ts, idxs=ΔΨm) sol2DM.(ts, idxs=ΔΨm)] .* 1000
deg = [sol2.(ts, idxs=degavg) sol2DM.(ts, idxs=degavg)]

In [ ]:
pl1 = plot(tsm, g3p, label=labels, ylabel="G3P (μM)", title="A", titlelocation=:left)
pl2 = plot(pyr, label=labels, ylabel="Pyruvate (μM)", title="B", titlelocation=:left)
pl3 = plot(tsm, nadh_c, label=labels, ylabel="Cytosolic NADH (μM)", title="C", titlelocation=:left)
pl4 = plot(tsm, nadh_m, label=labels, ylabel="Mitochondrial NADH (μM)", title="D", titlelocation=:left)
pl5 = plot(tsm, ca_c, label=labels, ylabel="Cytosolic Calcium (μM)", title="E", titlelocation=:left)
pl6 = plot(tsm, ca_m, label=labels, ylabel="Mitochondrial Calcium (μM)", title="F", titlelocation=:left)
pl7 = plot(tsm, td, label=labels, ylabel="ATP/ADP ratio", title="G", titlelocation=:left)
pl8 = plot(tsm, dpsi, label=labels, ylabel="Mitochodrial membrane potential (mV)", title="H", titlelocation=:left)
pl9 = plot(tsm, deg, label=labels, ylabel="Average Node Degree", title="I", titlelocation=:left)

plot(pl1, pl2, pl3, pl4, pl5, pl6, pl7, pl8, pl9; size=(2000, 2000))

In [ ]:
plot(sol2, vars=[J_O2])
plot!(sol2DM, vars=[J_O2])

In [ ]:
# Uncomment to generate PDF
# fig72.savefig("S1Fig3-2.pdf")